## Data Exploring, Cleaning and Feature Engineering

### Events

#### Overall

In [ ]:
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [ ]:
# Check the size of the dataset
len(events)

2756101

In [ ]:
# Check the data types of the columns
events.dtypes

timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

In [ ]:
# Check null values in the events DataFrame
events.isnull().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

In [ ]:
# Check if there are any duplicate entries in the events DataFrame
events[events.duplicated(keep='first')==True].shape[0]

460

In [ ]:
# Drop duplicates
events.drop_duplicates(keep='first', inplace=True)

#### transactionid

Looks like transctionid is not null only for certain events

In [ ]:
# Check transactionid non null values
events[~events['transactionid'].isnull()]

,timestamp,visitorid,event,itemid,transactionid
130,1433222276276,599528,transaction,356475,4000.0
304,1433193500981,121688,transaction,15335,11117.0
418,1433193915008,552148,transaction,81345,5444.0
814,1433176736375,102019,transaction,150318,13556.0
843,1433174518180,189384,transaction,310791,7244.0
...,...,...,...,...,...
2755294,1438377176570,1050575,transaction,31640,8354.0
2755349,1438379878779,861299,transaction,456602,3643.0
2755508,1438357730123,855941,transaction,235771,4385.0
2755603,1438355560300,548772,transaction,29167,13872.0


In [ ]:
# Check the reason for the non null values in transactionid
events[~events['transactionid'].isnull()]['event'].unique()

array(['transaction'], dtype=object)

In [ ]:
# Check if there are transactionid values that are null when the event is 'transaction'
events[events['event'] == 'transaction']['transactionid'].isnull().sum()

np.int64(0)

transactionid is not null only when the event is 'transaction'. For the dataset, transactionid is the amount spent in each event, so it makes sense to have it only when event is transaction

#### timestamp

Seems like timestamp is a Unix column. I am going to convert it to datetime

In [ ]:
# Convert 'timestamp' to datetime format and sort by 'timestamp'
events['timestamp'] = pd.to_datetime(events['timestamp'], unit='ms', origin='unix')
events.sort_values('timestamp', inplace=True)

In [ ]:
# Check time range of events
timedelta = (events['timestamp'].iloc[-1]-events['timestamp'].iloc[0]).days
timedelta

137

#### event

In [ ]:
# Check the type of events
events['event'].unique()

array(['addtocart', 'view', 'transaction'], dtype=object)

In [ ]:
print("Number of transactions:",events[events['event'] == 'transaction'].shape[0])
print("Number of views:",events[events['event'] == 'view'].shape[0])
print("Number of addtocart:",events[events['event'] == 'addtocart'].shape[0])


Number of transactions: 22457
Number of views: 2664218
Number of addtocart: 68966


#### visitorid

In [ ]:
# Check the number of unique visitors
events['visitorid'].unique().shape[0]

1407580

In [ ]:
# Check how many unique visitors actually made a purchase
events[events['event'] == 'transaction']['visitorid'].unique().shape[0]

11719

In [ ]:
# Check how many unique visitors added items to their cart
events[events['event'] == 'addtocart']['visitorid'].unique().shape[0]

37722

In [ ]:
# Check if the same visitorid appears in both 'addtocart' and 'transaction' events
events[(events['event'] == 'addtocart') & 
       (events['visitorid']
        .isin(events[events['event'] == 'transaction']['visitorid']))]['visitorid'].unique().shape[0]

10576

In [ ]:
# Check the visitorid that made a purchase but did not add items to their cart
events[(events['event'] == 'transaction') & 
       (events['visitorid']
        .isin(events[events['event'] == 'addtocart']['visitorid'])==False)]['visitorid'].unique().shape[0]

1143

In [ ]:
# Check the visitorid that made a purchase but did not viewed any items
events[(events['event'] == 'transaction') & 
       (events['visitorid']
        .isin(events[events['event'] == 'view']['visitorid'])==False)]['visitorid'].unique().shape[0]

428

In [ ]:
# Check the visitorid that made a purchase but did not viewed or added any items to their cart
events[(events['event'] == 'transaction') & 
       (events['visitorid']
        .isin(events[events['event'] == 'view']['visitorid'])==False) & 
       (events['visitorid']
        .isin(events[events['event'] == 'addtocart']['visitorid'])==False)]['visitorid'].unique().shape[0]

80

The visitorid that made purchases but did not viewed or added any items to their carts are strong candidates to be taken out of the dataset, since we cannot track their preferences and, therefore, train the bandits.

In [ ]:
# Calculate the number of interactions per visitor per day, considering only their active period

# Get the first and last interaction timestamp for each visitor
visitor_active_period = events.groupby('visitorid')['timestamp'].agg(['min', 'max'])

# Calculate the number of days each visitor was active (inclusive)
visitor_active_period['days_active'] = (visitor_active_period['max'] - visitor_active_period['min']).dt.days + 1

# Count total interactions per visitor
visitor_interactions = events.groupby('visitorid').size().rename('total_interactions')

# Merge to get both total interactions and days active
visitor_stats = visitor_active_period.join(visitor_interactions)

# Calculate average interactions per day during active period
visitor_stats['interactions_per_day'] = visitor_stats['total_interactions'] / visitor_stats['days_active']

# Filter visitors with more than 29 interactions and sort by interactions per day
visitor_stats[visitor_stats['total_interactions']>29].sort_values(by='interactions_per_day', ascending=False)

,min,max,days_active,total_interactions,interactions_per_day
visitorid,,,,,
530033,2015-06-04 14:55:58.341,2015-06-05 03:04:22.589,1,310,310.000000
1039026,2015-05-20 20:06:52.095,2015-05-21 02:13:22.333,1,284,284.000000
962131,2015-07-18 16:00:57.887,2015-07-19 15:01:36.400,1,267,267.000000
966491,2015-07-01 15:01:28.195,2015-07-02 02:28:23.676,1,250,250.000000
974226,2015-07-14 15:46:16.505,2015-07-17 03:11:16.204,3,694,231.333333
...,...,...,...,...,...
245403,2015-05-06 19:52:09.801,2015-09-09 03:47:26.643,126,30,0.238095
665707,2015-05-06 05:00:05.957,2015-09-10 04:27:18.343,127,30,0.236220
512411,2015-05-05 16:02:11.003,2015-09-14 15:45:12.246,132,31,0.234848


Visitors with less than 30 interactions are also strong candidates to be taken out of the dataset for the same reason above

#### itemid

In [ ]:
# Check the number of unique items
events['itemid'].unique().shape[0]

235061

In [ ]:
# Filter only events with itemid in properties
events = events[events['itemid'].isin(properties['itemid'])]

##### view

In [ ]:
# Check the number of unique items that were viewed
events[events['event'] == 'view']['itemid'].unique().shape[0]

185024

In [ ]:
# Check the items that represent 80% of the views
events[events['event'] == 'view']['itemid']\
    .value_counts(normalize=True)\
        .sort_values(ascending=False)\
            .cumsum()\
                .loc[lambda x: x <= 0.8]*100

itemid
187946     0.141497
461686     0.246810
5411       0.343285
370653     0.420216
219512     0.492417
            ...    
266039    79.997759
147964    79.998257
207474    79.998755
50250     79.999253
53807     79.999751
Name: proportion, Length: 40894, dtype: float64

##### addtocart

In [ ]:
# Check the number of unique items that were added to cart events
events[events['event'] == 'addtocart']['itemid'].unique().shape[0]

23458

In [ ]:
# Check the items that represent 80% of the added to cart events
events[events['event'] == 'addtocart']['itemid']\
    .value_counts(normalize=True)\
        .sort_values(ascending=False)\
            .cumsum()\
                .loc[lambda x: x <= 0.8]*100

itemid
461686     0.446186
312728     0.682489
409804     0.905582
320130     1.112530
29196      1.310672
            ...    
5790      79.986203
192472    79.989139
273940    79.992074
102434    79.995010
145298    79.997945
Name: proportion, Length: 11038, dtype: float64

##### transaction

In [ ]:
# Check the number of unique items that were purchased
events[events['event'] == 'transaction']['itemid'].unique().shape[0]

11645

In [ ]:
# Check the items that represent 80% of the purchases
events[events['event'] == 'transaction']['itemid']\
    .value_counts(normalize=True)\
        .sort_values(ascending=False)\
            .cumsum()\
                .loc[lambda x: x <= 0.8]*100

itemid
461686     0.605040
119736     1.046311
213834     1.464835
312728     1.674097
7943       1.883359
            ...    
265780    79.979074
135174    79.983623
134906    79.988172
146297    79.992721
114514    79.997270
Name: proportion, Length: 7248, dtype: float64

3% of the registered products represent 80% of all purchases. Those are the strongest candidates to be recommended

### Recommended items and user picking

#### Users

In [ ]:
# Get the users that interacted with the website at least 10 times
users = visitor_stats[visitor_stats['total_interactions']>9].index.tolist()

In [ ]:
# Remove the visitorid that made a purchase but did not viewed or added any items to their cart
users = pd.Series(users)[~pd.Series(users).isin(
    events[
        (events['event'] == 'transaction') &
        (~events['visitorid'].isin(events[events['event'] == 'view']['visitorid'])) &
        (~events['visitorid'].isin(events[events['event'] == 'addtocart']['visitorid']))
    ]['visitorid'].unique()
)]
users = users.tolist()

In [ ]:
# Filter the events DataFrame to only include the users that interacted with the website at least 30 times
events = events[events['visitorid'].isin(users)]

#### Items

In [ ]:
# Store the most representative purchased items
rec_items = events[events['event'] == 'transaction']['itemid']\
    .value_counts(normalize=True)\
        .sort_values(ascending=False)\
            .cumsum()\
                .loc[lambda x: x <= 0.8].index.tolist()

There might be some items that were purchased but not viewed or added to cart. These could be taken out of the recommended items list as we want optimize the user interaction in all ways with the purchased items

In [ ]:
# Check if all items that were purchased were also viewed or added to cart
purchased_notviewed_notadded = events[((events['event'] == 'transaction') & 
       (events['itemid']
        .isin(events[events['event'] == 'view']['itemid'])==False)) | 
       ((events['event'] == 'transaction') & 
       (events['itemid']
        .isin(events[events['event'] == 'addtocart']['itemid'])==False))]['itemid'].unique()


In [ ]:
# Check how many items in the purchased_notviewed_notadded list are in the rec_items list
np.isin(purchased_notviewed_notadded, rec_items).sum()

np.int64(150)

In [ ]:
# Remove the items that were purchased but not viewed or added to cart from the rec_items list
rec_items = [itm for itm in rec_items if itm not in purchased_notviewed_notadded]

In [ ]:
len(rec_items)

5426

### Properties

In [ ]:
properties

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
20275897,1433646000000,236931,929,n12.000
20275898,1440903600000,455746,6,150169 639134
20275899,1439694000000,347565,686,610834
20275900,1433646000000,287231,867,769062


In [ ]:
properties['itemid'].unique().shape[0]

417053

In [ ]:
properties[properties['property'] == 'categoryid']['itemid'].unique().shape[0]

417053

In [ ]:
# Filter the properties DataFrame to only include items in the rec_items list
# properties = properties[properties['itemid'].isin(rec_items)]

In [ ]:
# Check the number of items in the properties DataFrame
properties['itemid'].unique().shape[0]

417053

In [ ]:
properties = properties[properties['itemid'].isin(events['itemid'].unique())]

In [ ]:
properties['itemid'].unique().shape[0]

72386

In [ ]:
# Convert 'timestamp' to datetime format
properties['timestamp'] = pd.to_datetime(properties['timestamp'], unit='ms', origin='unix')

# Filter properties so that for each itemid, only property rows with timestamp <= latest event timestamp for that itemid are kept

# Get the latest event timestamp for each itemid
#latest_event_ts = events.groupby('itemid')['timestamp'].max()

# Map the latest event timestamp to the properties DataFrame
#properties['latest_event_ts'] = properties['itemid'].map(latest_event_ts)

# Keep only property rows where the property timestamp is less than or equal to the latest event timestamp for that itemid
#properties = properties[properties['timestamp'] <= properties['latest_event_ts']].drop(columns='latest_event_ts')

# Filter properties so that for each itemid, only property rows with maximum timestamp before the earliest event timestamp for that itemid are kept

# Get the earliest event timestamp for each itemid
#earliest_event_ts = events.groupby('itemid')['timestamp'].min()

# Map the earliest event timestamp to the properties DataFrame
#properties['earliest_event_ts'] = properties['itemid'].map(earliest_event_ts)

# Keep only property rows where the property timestamp is the maximum before the earliest event or equal to the earliest event timestamp for that itemid
#properties = properties[(properties['timestamp'] < properties['earliest_event_ts']) | 
#                         (properties['timestamp'] == properties['earliest_event_ts'])]\
#                            .drop(columns='earliest_event_ts')

C:\Users\fldam\AppData\Local\Temp\ipykernel_12384\12958538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties['timestamp'] = pd.to_datetime(properties['timestamp'], unit='ms', origin='unix')


In [ ]:
properties.sort_values(by=['itemid', 'timestamp'])

,timestamp,itemid,property,value
562977,2015-05-10 03:00:00,6,categoryid,1091
4412612,2015-05-10 03:00:00,6,888,609354
5900583,2015-05-10 03:00:00,6,764,1285872
6624891,2015-05-10 03:00:00,6,available,1
12499370,2015-05-10 03:00:00,6,790,n44040.000
...,...,...,...,...
13407693,2015-09-06 03:00:00,466864,available,0
2238896,2015-09-13 03:00:00,466864,790,n111840.000
5763096,2015-09-13 03:00:00,466864,813,1148082 353870 1262739
12885429,2015-09-13 03:00:00,466864,888,1262739 205682 1050016 1154859


In [ ]:
# Separate values starting with 'n' from the others in props, process as requested
props = properties['value'].str.split()

cat_props_list = []
num_props_list = []

for prop_list in props:
    cat_props = []
    num_props = []
    if isinstance(prop_list, list):
        for p in prop_list:
            if p.startswith('n'):
                try:
                    num_props.append(float(p[1:]))
                except ValueError:
                    continue
            else:
                try:
                    cat_props.append(int(p))
                except ValueError:
                    continue
    cat_props_list.append(cat_props)
    num_props_list.append(num_props)

# Add as new columns to properties DataFrame
properties['cat_props'] = cat_props_list
properties['num_props'] = num_props_list
properties.drop(columns='value', inplace=True)

C:\Users\fldam\AppData\Local\Temp\ipykernel_12384\2059359439.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties['cat_props'] = cat_props_list
C:\Users\fldam\AppData\Local\Temp\ipykernel_12384\2059359439.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties['num_props'] = num_props_list
C:\Users\fldam\AppData\Local\Temp\ipykernel_12384\2059359439.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [ ]:
properties[properties['num_props'].apply(lambda x: len(x) > 0)]

,timestamp,itemid,property,cat_props,num_props
1,2015-09-06 03:00:00,206783,888,"[1116713, 960601]",[277.2]
3,2015-05-10 03:00:00,59481,790,[],[15360.0]
11,2015-06-28 03:00:00,244127,400,"[639502, 424566]","[552.0, 720.0]"
14,2015-06-14 03:00:00,169055,790,[],[21000.0]
20,2015-08-16 03:00:00,48696,566,"[639502, 189174]",[480.0]
...,...,...,...,...,...
20275798,2015-06-07 03:00:00,140861,888,[1133979],[13308.0]
20275805,2015-05-31 03:00:00,305760,19,"[1297729, 1749, 1178208, 350726]","[72.0, 36.0]"
20275860,2015-07-19 03:00:00,241233,790,[],[12048.0]
20275862,2015-08-23 03:00:00,358049,888,"[1320974, 56529, 237874, 583354, 1297729, 3507...",[96.0]


In [ ]:
properties[properties['cat_props'].apply(lambda x: len(x) > 0)]

,timestamp,itemid,property,cat_props,num_props
1,2015-09-06 03:00:00,206783,888,"[1116713, 960601]",[277.2]
11,2015-06-28 03:00:00,244127,400,"[639502, 424566]","[552.0, 720.0]"
19,2015-06-28 03:00:00,363598,1022,"[857891, 593337]",[]
20,2015-08-16 03:00:00,48696,566,"[639502, 189174]",[480.0]
22,2015-06-14 03:00:00,269797,159,[519769],[]
...,...,...,...,...,...
20275875,2015-08-16 03:00:00,4849,888,"[297765, 406279, 907471, 88645, 279913, 122335...",[]
20275877,2015-06-07 03:00:00,279551,348,[530843],[]
20275882,2015-06-28 03:00:00,10107,505,[769062],[]
20275891,2015-08-09 03:00:00,200211,available,[0],[]


In [ ]:
properties_grouped = properties[~properties['property'].isin(['available', 'categoryid'])].groupby(['timestamp', 'itemid']).agg({
    'property': list
}).reset_index()

In [ ]:
properties_grouped.itemid.unique()

array([     6,     15,     16, ..., 462265, 463360, 465950],
      shape=(72386,))

#### Pre-processing properties

In [ ]:
# Group properties by 'timestamp' and 'itemid', and create a dictionary mapping each property to its cat_props and num_props
def property_dict(row):
    return {
        prop: {'cat_props': cat, 'num_props': num}
        for prop, cat, num in zip(row['property'], row['cat_props'], row['num_props'])
    }

# First, group and aggregate lists for each column
properties_grouped = properties[~properties['property'].isin(['available', 'categoryid'])].groupby(['timestamp', 'itemid']).agg({
    'property': list,
    'cat_props': list,
    'num_props': list
}).reset_index()

# Now, create the dictionary column
properties_grouped['property_dict'] = properties_grouped.apply(property_dict, axis=1)

# Show the resulting DataFrame with the new dictionary column
properties_grouped[['timestamp', 'itemid', 'property_dict']].head()

KeyboardInterrupt: 

In [ ]:
properties_grouped = properties_grouped[['timestamp', 'itemid', 'property_dict']]

In [ ]:
# Timestamp mínimo por itemid na tabela de propriedades
min_ts_props = properties_grouped.groupby('itemid')['timestamp'].min()

# Merge para trazer a data mínima das propriedades para os eventos
events_filtered = events.merge(
    min_ts_props.rename('min_prop_ts'), 
    on='itemid', 
    how='left'
)

# Filtra eventos ocorridos após a primeira propriedade do item
events_filtered = events_filtered[events_filtered['timestamp'] >= events_filtered['min_prop_ts']]
events_filtered = events_filtered.drop(columns='min_prop_ts')

In [ ]:
# Hashing dimensionality — ajustável com base na complexidade do modelo
HASH_SIZE = 512
MAX_NUM_PROPS = 20  # limite de valores numéricos por item (pode ajustar)

# Inicializa o hasher
hasher = FeatureHasher(n_features=HASH_SIZE, input_type='string')

def process_property_dict(property_dict):
    cat_tokens = []
    num_values = []

    for prop_name, values in property_dict.items():
        # Ignora propriedades indesejadas
        if prop_name in ['categoryid', 'available']:
            continue

        # Categorias: gerar tokens como '790=1047026'
        cat_props = values.get('cat_props', [])
        for val in cat_props:
            cat_tokens.append(f"{prop_name}={val}")
        
        # Numéricos: adiciona diretamente
        num_values.extend(values.get('num_props', []))

    # Vetor hashing das categorias
    hashed_vec = hasher.transform([cat_tokens]).toarray()[0]

    # Normaliza o número de num_props
    num_values = (num_values + [0.0] * MAX_NUM_PROPS)[:MAX_NUM_PROPS]
    
    # Vetor final de contexto
    context_vector = np.concatenate([hashed_vec, np.array(num_values)])

    return context_vector

In [ ]:
properties_grouped['context'] = properties_grouped['property_dict'].apply(process_property_dict)

In [ ]:
# Ordena para merge_asof
events_filtered = events_filtered.sort_values(['timestamp', 'itemid'])
properties_grouped = properties_grouped.sort_values(['timestamp', 'itemid'])

# Junta o contexto mais recente antes de cada evento
df_merged = pd.merge_asof(
    events_filtered,
    properties_grouped[['itemid', 'timestamp', 'context']],
    on='timestamp',
    by='itemid',
    direction='backward'
)

In [ ]:
# Check for NaN values in the 'context' column of df_merged
df_merged['context'].isnull().sum()

np.int64(0)

In [ ]:
# Merge properties with categories to get the category names
#properties = properties.merge(categories, left_on='property', right_on='categoryid', how='left')
# Explode the cat_props and num_props columns
#properties = properties.explode('cat_props').explode('num_props')
# Merge properties with categories to get the category names
#properties = properties.merge(categories, left_on='cat_props', right_on='categoryid', how='left')
# Drop the 'categoryid' column as it's no longer needed
#properties.drop(columns='categoryid', inplace=True)
# Rename the columns for clarity
#properties.rename(columns={'cat_props': 'categoryid', 'name': 'category_name'}, inplace=True)

In [ ]:
# Explode cat_props_list into a flat list of category ids
#flat_cat_props = pd.Series(cat_props_list).explode().dropna().astype(int)
#flat_cat_props.drop_duplicates(inplace=True)
#flat_cat_props

### Categories

In [ ]:
categories.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [ ]:
len(categories)

1669

In [ ]:
# Check the data types of the columns
categories.dtypes

categoryid      int64
parentid      float64
dtype: object

In [ ]:
# Check null values in the categories DataFrame
categories.isnull().sum()

categoryid     0
parentid      25
dtype: int64

In [ ]:
rootcat = categories[categories['parentid'].isnull()]
firstgencat = categories[categories['parentid'].isin(rootcat['categoryid'])]
secondgencat = categories[categories['parentid'].isin(firstgencat['categoryid'])]
thirdgencat = categories[categories['parentid'].isin(secondgencat['categoryid'])]
fourthgencat = categories[categories['parentid'].isin(thirdgencat['categoryid'])]
fifthgencat = categories[categories['parentid'].isin(fourthgencat['categoryid'])]

In [ ]:
len(rootcat[~rootcat['categoryid'].isin(firstgencat['parentid'])])
#len(firstgencat[~firstgencat['categoryid'].isin(secondgencat['parentid'])])
#len(secondgencat[~secondgencat['categoryid'].isin(thirdgencat['parentid'])])
#len(thirdgencat[~thirdgencat['categoryid'].isin(fourthgencat['parentid'])])
#len(fourthgencat[~fourthgencat['categoryid'].isin(fifthgencat['parentid'])])

1

In [ ]:
len(categories['categoryid'].unique())

1669

In [ ]:
len(rootcat)+len(firstgencat)+len(secondgencat)+len(thirdgencat)+len(fourthgencat)+len(fifthgencat)

1669

In [ ]:
root = rootcat.rename(columns={'categoryid': 'root'}).drop(columns='parentid')
layer_1 = firstgencat.rename(columns={'categoryid': 'layer_1', 'parentid': 'root'})
layer_2 = secondgencat.rename(columns={'categoryid': 'layer_2', 'parentid': 'layer_1'})
layer_3 = thirdgencat.rename(columns={'categoryid': 'layer_3', 'parentid': 'layer_2'})
layer_4 = fourthgencat.rename(columns={'categoryid': 'layer_4', 'parentid': 'layer_3'})
layer_5 = fifthgencat.rename(columns={'categoryid': 'layer_5', 'parentid': 'layer_4'})

In [ ]:
tree_l5 = layer_5.merge(layer_4, on='layer_4').merge(layer_3, on='layer_3').merge(layer_2, on='layer_2').merge(layer_1, on='layer_1').merge(root, on='root')
tree_l4 = layer_4.merge(layer_3, on='layer_3').merge(layer_2, on='layer_2').merge(layer_1, on='layer_1').merge(root, on='root')
tree_l3 = layer_3.merge(layer_2, on='layer_2').merge(layer_1, on='layer_1').merge(root, on='root')
tree_l2 = layer_2.merge(layer_1, on='layer_1').merge(root, on='root')
tree_l1 = layer_1.merge(root, on='root')
cat_tree = pd.concat([tree_l1, tree_l2, tree_l3, tree_l4, tree_l5], ignore_index=True)

In [ ]:
neworder = ['root','layer_1','layer_2','layer_3','layer_4','layer_5']
cat_tree = cat_tree.reindex(neworder, axis=1).sort_values(by=neworder)
cat_tree

,root,layer_1,layer_2,layer_3,layer_4,layer_5
1151,140.0,61.0,323.0,1558.0,NaN,NaN
760,140.0,61.0,323.0,NaN,NaN,NaN
1153,140.0,61.0,897.0,120.0,NaN,NaN
1007,140.0,61.0,897.0,1098.0,NaN,NaN
1528,140.0,61.0,897.0,1317.0,NaN,NaN
...,...,...,...,...,...,...
732,1698.0,1678.0,305.0,NaN,NaN,NaN
730,1698.0,1678.0,455.0,NaN,NaN,NaN
779,1698.0,1678.0,1346.0,NaN,NaN,NaN
731,1698.0,1678.0,1554.0,NaN,NaN,NaN


In [ ]:
cat_tree[cat_tree['root']==250]

,root,layer_1,layer_2,layer_3,layer_4,layer_5
1,250.0,7.0,NaN,NaN,NaN,NaN
67,250.0,74.0,NaN,NaN,NaN,NaN
64,250.0,238.0,NaN,NaN,NaN,NaN
149,250.0,329.0,NaN,NaN,NaN,NaN
72,250.0,391.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
359,250.0,1669.0,814.0,NaN,NaN,NaN
634,250.0,1669.0,1226.0,NaN,NaN,NaN
357,250.0,1669.0,1440.0,NaN,NaN,NaN
37,250.0,1669.0,NaN,NaN,NaN,NaN


In [ ]:
# Check if the total number of categories matches the sum of all category levels
len(categories) == len(rootcat) + len(firstgencat) + len(secondgencat) + len(thirdgencat) + len(fourthgencat) + len(fifthgencat)

True

In [ ]:
categories['parentid'].unique().size

363